In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

# Task 1

For easier repeating process for different countries, a function is defined

Check the parameters and instructions in the function

The sample below the function is also the solution for the task 1

For the output excel, I keep the "Gov Health Expenditure (% of Total Health Exp)" beyond 1 decimal point because it is from the original data.

In [3]:
def analyze_health_expenditure(file_path, selected_countries=None, top_n=10):
    """
    Analyzes year-over-year growth in government health expenditure for selected countries.
    
    Parameters:
        file_path (str): Path to the Excel file containing data.
        selected_countries (list): List of countries to analyze. If None, selects top N populated countries.
        top_n (int): Number of most populated countries to select if selected_countries is None.
    
    Instructions:
        1. Set file_path parameter as the path to the Excel file containing data.
        2. If specific countries are intrested, provide a list of countries names as selected_countries parameter. Note: Please ensure that the country names are spelled correctly and match the data in the Excel file.
        3. If selected_countries is None, the function will select the top N most populated countries in 2015, in this case, set the top_n parameter.
        4. Run the function and the formatted table will be printed along with the report saved in an Excel file.

    Returns:
        None - Prints formatted table and saves report to an Excel file.
    """
    # Load the Excel file
    xls = pd.ExcelFile(file_path)

    # Load and clean population data
    df_population = pd.read_excel(xls, sheet_name='Data-POPLN', skiprows=3)
    df_population = df_population.rename(columns={"Country Name": "Country", "Country Code": "Country Code"})
    df_population = df_population.drop(columns=["Indicator Name", "Indicator Code"])
    df_population = df_population.melt(id_vars=["Country", "Country Code"], var_name="Year", value_name="Population")
    df_population["Year"] = df_population["Year"].astype(int)

    # Load metadata to filter required countries
    df_meta = pd.read_excel(xls, sheet_name='Meta-POPLN')
    df_meta = df_meta.rename(columns={"TableName": "Country"})
    df_meta_filtered = df_meta[(df_meta["Region"] == "East Asia & Pacific") & (df_meta["IncomeGroup"] == "Lower middle income")]

    # Determine selected countries
    if selected_countries is None:
        # If no countries are provided, select the top N most populated ones in 2015
        df_population_2015 = df_population[df_population["Year"] == 2015]
        selected_countries = df_population_2015[df_population_2015["Country"].isin(df_meta_filtered["Country"])] \
            .nlargest(top_n, "Population")["Country"].tolist()

    # Load and clean health expenditure data
    df_health_expenditure = pd.read_excel(xls, sheet_name='Data-HLTHEXPNDITURE', skiprows=3)
    df_health_expenditure.columns = df_health_expenditure.iloc[0]
    df_health_expenditure = df_health_expenditure[1:].reset_index(drop=True)
    df_health_expenditure = df_health_expenditure.dropna(axis=1, how="all")
    df_health_expenditure = df_health_expenditure.rename(columns={"Country Name": "Country", "Country Code": "Country Code"})

    # Reshape health expenditure data
    df_health_expenditure = df_health_expenditure.melt(id_vars=["Country", "Country Code"], var_name="Year", value_name="Health Expenditure")
    df_health_expenditure = df_health_expenditure[~df_health_expenditure["Year"].isin(["Series Name", "Series Code"])]
    df_health_expenditure["Year"] = df_health_expenditure["Year"].str.extract(r"(\d{4})").astype(int)
    df_health_expenditure["Health Expenditure"] = pd.to_numeric(df_health_expenditure["Health Expenditure"], errors='coerce')

    # Filter only selected countries
    df_health_expenditure = df_health_expenditure[df_health_expenditure["Country"].isin(selected_countries)]

    # Compute year-over-year percentage increase
    df_health_expenditure = df_health_expenditure.sort_values(by=["Country", "Year"])
    df_health_expenditure["YoY Increase (%)"] = df_health_expenditure.groupby("Country")["Health Expenditure"].pct_change() * 100

    # Filter for the years 2005-2015
    df_health_expenditure_filtered = df_health_expenditure[(df_health_expenditure["Year"] >= 2005) & (df_health_expenditure["Year"] <= 2015)]
    df_health_expenditure_filtered["YoY Increase (%)"] = df_health_expenditure_filtered["YoY Increase (%)"].round(1)

    # Rename columns for presentation
    df_final = df_health_expenditure_filtered.rename(columns={
        "Country": "Country Name",
        "Year": "Year",
        "Health Expenditure": "Gov Health Expenditure (% of Total Health Exp)",
        "YoY Increase (%)": "Year-over-Year Growth (%)"
    })

    # Print a formatted table
    print(tabulate(df_final, headers='keys', tablefmt='pretty'))

    # Save results to an Excel file
    df_final.to_excel("Task1_report.xlsx", index=False)

    print("Analysis completed. The formatted report is saved in 'Task1_report.xlsx'.")

In [4]:
# Example Usage:
# 
analyze_health_expenditure("Task1_Analyze.xlsx", selected_countries=None)

+------+------------------+--------------+------+------------------------------------------------+---------------------------+
|      |   Country Name   | Country Code | Year | Gov Health Expenditure (% of Total Health Exp) | Year-over-Year Growth (%) |
+------+------------------+--------------+------+------------------------------------------------+---------------------------+
| 1552 |     Cambodia     |     KHM      | 2005 |                  17.83416748                   |           -8.1            |
| 1769 |     Cambodia     |     KHM      | 2006 |                  15.69289398                   |           -12.0           |
| 1986 |     Cambodia     |     KHM      | 2007 |                  19.81499672                   |           26.3            |
| 2203 |     Cambodia     |     KHM      | 2008 |                  15.53217125                   |           -21.6           |
| 2420 |     Cambodia     |     KHM      | 2009 |                  16.06786346                   |            3